In [1]:
import time
from scrapy.crawler import CrawlerProcess
import pandas as pd
import numpy as np
from collections import Counter
from scraping import *

In [2]:
pop_df = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/most_pop_nodes.csv")

In [3]:
N = len(pop_df)#50

nodes = list(pop_df["index"][:N])

In [4]:
nodes = [
            '03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d'
        ]

In [6]:
#spider = LNNodeParser(nodes=nodes)
process = CrawlerProcess(LNNodeParser)
process.crawl()
process.start()

AttributeError: type object 'LNNodeParser' has no attribute 'get'

In [ ]:
process.stop()

In [ ]:
meta_records = []
for i, n in enumerate(nodes):
    meta_records.append(extract_node_meta_data(n))
    print(i)

In [ ]:
len(meta_records)

meta_records

In [ ]:
meta_df = pd.DataFrame(meta_records)

In [ ]:
meta_df.head()

meta_df.loc[8]["pub_key"]

In [ ]:
(meta_df.isnull().sum() / N).sort_values()

In [ ]:
meta_df.to_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_meta.csv", index=False)

In [ ]:
meta_with_labels_df = meta_df[~meta_df["labels"].isnull()]

In [ ]:
meta_with_labels_df["num_labels"] = meta_with_labels_df["labels"].apply(len)

In [ ]:
meta_with_labels_df["num_labels"].value_counts()

In [ ]:
len(meta_with_labels_df)

In [ ]:
meta_with_labels_df.to_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_meta_with_labels.csv", index=False)

In [ ]:
all_labels = []
for lab_list in list(meta_with_labels_df["labels"]):
    all_labels += lab_list

In [ ]:
cnt = Counter(all_labels)

In [ ]:
meta_with_labels_df.head(20)

In [ ]:
len(cnt), len(meta_df)

### Label merging ideas:
  
- G1: every kind of games
- G2: Banking, Excange (not G1)
- G3: Wallet (not G1 or G2)
- G4: every kind of Store (not G1, G2, G3)
- G5: all remaining service providers (not G1, G2, G3, G4)

Ami csak 'Open Source' vagy 'Testing' azokat nem venném figyelembe

In [ ]:
cnt.most_common()

In [ ]:
label_pos = dict(zip(cnt.keys(), range(len(cnt))))

In [ ]:
pivot_records = []
onehot_labels = np.zeros((len(meta_with_labels_df), len(cnt)))
for idx, labels in enumerate(list(meta_with_labels_df["labels"])):
    for lab in labels:
        pivot_records.append([idx, lab])
        onehot_labels[idx, label_pos[lab]] = 1

In [ ]:
onehot_labels_df = pd.DataFrame(onehot_labels, columns=cnt.keys())

In [ ]:
pivot_records = pd.DataFrame(pivot_records, columns=["item","label"])
pivot_records.head()

In [ ]:
onehot_labels_df[onehot_labels_df['Internet of Things'] == 1].sum()

In [ ]:
onehot_labels_df[onehot_labels_df['Banking'] == 1].sum()